In [1]:
import cv2
import mediapipe as mp
import numpy as np
import csv
from collections import deque

In [ ]:


# Initialize the video capture and MediaPipe Pose
cap = cv2.VideoCapture(0)
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Curl counter variables for both arms
counter_left = 0
counter_right = 0
stage_left = None
stage_right = None
angles_left = deque(maxlen=5)  # To store the last 5 angles for smoothing (left arm)
angles_right = deque(maxlen=5)  # To store the last 5 angles for smoothing (right arm)

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Second point
    c = np.array(c)  # Third point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

# Save workout data to a CSV file
def save_workout_data(counter_left, counter_right):
    with open('workout_data.csv', 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([counter_left, counter_right])

# Mediapipe Pose Detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for the left arm
            shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle for the left arm
            angle_left = calculate_angle(shoulder_left, elbow_left, wrist_left)
            angles_left.append(angle_left)
            smoothed_angle_left = np.mean(angles_left)  # Smoothed angle using the moving average
            
            # Curl counter logic for the left arm
            if smoothed_angle_left > 160:
                stage_left = "d"
            if smoothed_angle_left < 30 and stage_left == 'd':
                stage_left = "up"
                counter_left += 1
                save_workout_data(counter_left, counter_right)  # Save the rep count to the file
            
            # Get coordinates for the right arm
            shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle for the right arm
            angle_right = calculate_angle(shoulder_right, elbow_right, wrist_right)
            angles_right.append(angle_right)
            smoothed_angle_right = np.mean(angles_right)  # Smoothed angle using the moving average
            
            # Curl counter logic for the right arm
            if smoothed_angle_right > 160:
                stage_right = "d"
            if smoothed_angle_right < 30 and stage_right == 'd':
                stage_right = "up"
                counter_right += 1
                save_workout_data(counter_left, counter_right)  # Save the rep count to the file

        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (480,73),(236,110,202), -1)
        
        # Rep data for the left arm
        cv2.putText(image, 'REPS LEFT', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter_left), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data for the left arm
        cv2.putText(image, ' LEFT', (150,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage_left, 
                    (145,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Rep data for the right arm
        cv2.putText(image, 'REPS RIGHT', (250,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter_right), 
                    (245,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data for the right arm
        cv2.putText(image, ' RIGHT', (380,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage_right, 
                    (375,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(210,135,42), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(252,196,248), thickness=2, circle_radius=2))
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
# Ajout de la correction de posture pour le bras gauche
if smoothed_angle_left > 160:
    stage_left = "down"
elif smoothed_angle_left > 90:  # Ajoute une vérification si l'utilisateur ne va pas assez bas
    cv2.putText(image, 'Descend plus bas avec le bras gauche!', (50,150), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2, cv2.LINE_AA)

# Ajout de la correction de posture pour le bras droit
if smoothed_angle_right > 160:
    stage_right = "down"
elif smoothed_angle_right > 90:  # Vérification similaire pour le bras droit
    cv2.putText(image, 'Descend plus bas avec le bras droit!', (50,200), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2, cv2.LINE_AA)

    cap.release()
    cv2.destroyAllWindows()

C:\Users\sirin\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
